In [2]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Google sheet - Get handle on it

A Google sheet is used to store the names of image files as well as the text transcribed from handwriting.

In [ ]:
!pip install --upgrade -q gspread

In [ ]:
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)

sh = gc.open('fieldbook_pages')
worksheet_pages = sh.worksheet(title="pages")
worksheet_contrasted_pages = sh.worksheet(title="contrasted_pages")

# Contrast images

In [ ]:
# list images in a directory
import os 
for x in os.listdir('/content/drive/MyDrive/fieldbooks_images/1932_road_to_aleppo_12'):
    if x.endswith(".jpg"):
        # Prints only text file present in My Folder
        print(x)

In [ ]:
# convert gray images to black and white
import cv2 
from google.colab.patches import cv2_imshow

import os

raw_images_dir = '/content/drive/MyDrive/fieldbooks_images/1932_road_to_aleppo_12/'
contrast_images_dir = os.path.join(raw_images_dir,"contrasted/") 
for raw_image_file_name in os.listdir(raw_images_dir):
    if raw_image_file_name.endswith(".jpg"):
        print(raw_image_file_name)
        # Prints only text file present in My Folder

        raw_img = cv2.imread(os.path.join(raw_images_dir, raw_image_file_name))
        gray_image = cv2.cvtColor(raw_img, cv2.COLOR_BGR2GRAY)
        #https://pyimagesearch.com/2021/04/28/opencv-thresholding-cv2-threshold/
        #blurred_gray_image = cv2.GaussianBlur(gray_image, (3, 3), 0) -- didn't really work here
        (thresh, binary) = cv2.threshold(gray_image, 208, 255, cv2.THRESH_BINARY)
        #cv2_imshow(img)
        cv2_imshow(gray_image)
        cv2.imwrite(os.path.join(contrast_images_dir,raw_image_file_name), binary)
#cv2.waitKey(0)
#cv2.destroyAllWindows()

In [ ]:
import cv2 
import numpy as np 

img =   cv2.imread("/content/drive/MyDrive/fieldbooks_images/1932_road_to_aleppo_12/ANT_FB_1932-003-0000.jpg")[...,0]
#bgray = cv2.imread(fname)[...,0]

blured1 = cv2.medianBlur(img,3)
blured2 = cv2.medianBlur(img,51)
divided = np.ma.divide(blured1, blured2).data
normed = np.uint8(255*divided/divided.max())
th, threshed = cv2.threshold(normed, 100, 255, cv2.THRESH_OTSU)

dst = np.vstack((img, blured1, blured2, normed, threshed)) 
cv2_imshow(dst)

NameError: ignored

# Azure Cognitive Services. 
+ Authenticate and Connect
+ For each link in the spreadsheet, transcribe and store in sheet

In [15]:
# Get keys to connect
import os
import pandas as pd
import json

params_directory = "/content/drive/MyDrive/params/" # Change this to suit your machine

params_file_path = os.path.join(params_directory,"params_acs.json")

# Verify the configuration file exists
if(os.path.exists(params_file_path)):
    print("Success.",params_file_path, "exists.")
else:
    print("Failure.",params_file_path, "does not exist.")

# Read the JSON file into a DataFrame
params_dict = pd.read_json(params_file_path, typ='dictionary')

# Store as enivonmental variables
acs_subscription_key = params_dict['COMPUTER_VISION_SUBSCRIPTION_KEY']
acs_endpoint = params_dict['COMPUTER_VISION_ENDPOINT']

# Do some basic validation
if len(acs_subscription_key) == 32:
    print("Success. COMPUTER_VISION_SUBSCRIPTION_KEY is loaded.")
else:
    print("Error. The COMPUTER_VISION_SUBSCRIPTION_KEY is not the expected length, please check it.")

Success. /content/drive/MyDrive/params/params_acs.json exists.
Success. COMPUTER_VISION_SUBSCRIPTION_KEY is loaded.


In [10]:
# Install what is required to connect to Azure Cognitive Services Computer Vision
# Run this once on your machine. If you are using Google Colab, run this once per session.
!pip install --upgrade azure-cognitiveservices-vision-computervision

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 85 kB 3.1 MB/s 
     |████████████████████████████████| 41 kB 573 kB/s 
     |████████████████████████████████| 178 kB 37.6 MB/s 


In [11]:
# Run this once per session 

# Import the required libraries
from azure.cognitiveservices.vision.computervision import ComputerVisionClient
from azure.cognitiveservices.vision.computervision.models import OperationStatusCodes
from azure.cognitiveservices.vision.computervision.models import VisualFeatureTypes
from msrest.authentication import CognitiveServicesCredentials
import sys

# Authenticate with Azure Cognitive Services.
computervision_client = ComputerVisionClient(acs_endpoint, CognitiveServicesCredentials(acs_subscription_key))

In [12]:
# function to trascribe an image stored on a drive
import time

def get_text_from_image_acs(read_image_path):
    # Open the image
    read_image = open(read_image_path, "rb")
    # Call API with image and raw response (allows you to get the operation location). Call Azure using computervision_client with the image.
    read_response = computervision_client.read_in_stream(read_image, raw=True)
    # Get the operation location (URL with ID as last appendage)
    read_operation_location = read_response.headers["Operation-Location"]
    # Take the ID off and use to get results
    operation_id = read_operation_location.split("/")[-1]
    # Call the "GET" API and wait for the retrieval of the results
    while True:
        read_result = computervision_client.get_read_result(operation_id)
        if read_result.status.lower () not in ['notstarted', 'running']:
            break
            print ('Waiting for result...')
            time.sleep(10)

    # Print results, line by line
    return_value = ""
    if read_result.status == OperationStatusCodes.succeeded:
        for text_result in read_result.analyze_result.read_results:
            for line in text_result.lines:
                return_value = return_value + str(line.text) + "\n"  
                print(line.text)
                print(line.bounding_box)
    return(return_value)

In [13]:
# test the method
text_on_image = get_text_from_image_acs("/content/drive/MyDrive/fieldbooks_images/1932_road_to_aleppo_12/contrasted/ANT_FB_1932-003-0000.jpg")
print(text_on_image)

1 Juin.
[126.0, 29.0, 215.0, 29.0, 214.0, 60.0, 125.0, 61.0]
C'est un terrain situé à l'Est or la Route ; il est conlyn
[68.0, 87.0, 647.0, 89.0, 646.0, 119.0, 68.0, 117.0]
l'un 1 derrières massimo ; plante de quelpas figuiers - dal
[67.0, 133.0, 633.0, 131.0, 633.0, 159.0, 67.0, 161.0]
acting but jeers . it off presseurs ; in ya reme'de mai's.
[64.0, 172.0, 634.0, 172.0, 634.0, 198.0, 64.0, 202.0]
Plusieurs tuns voision, d'environ 5m de profondeur, as servi
[66.0, 214.0, 657.0, 213.0, 657.0, 244.0, 66.0, 246.0]
À carrier ; men a extrait is press dailles, with fragment
[66.0, 260.0, 671.0, 250.0, 672.0, 280.0, 67.0, 286.0]
gardent belle apparence. Les travaux net avéli pas 11. Pust
[64.0, 299.0, 689.0, 295.0, 690.0, 326.0, 64.0, 330.0]
De puits voisin A la maison voisine . for nouveau
[131.0, 341.0, 633.0, 341.0, 633.0, 372.0, 131.0, 372.0]
Sean : - 11mbo . I st use seemite love to fouls .
[72.0, 378.0, 661.0, 375.0, 661.0, 409.0, 72.0, 413.0]
None offirms & droit as faul un povagy, è 

# Transcribe images stored on Google Drive.
The images on the website are written in pencil and are difficult to read due to the low contrast between the color of pencil lead and paper. These images were processed to enhance the contrast by rendering them into black and white images.

Original:

https://ochre.lib.uchicago.edu/ochre?uuid=c14ddaa8-c833-42cc-be0b-a89624f1302c&load

Processed:

https://jeffblackadar.ca/fieldbook_antioch/ANT_FB_1932-003-0000.jpg

In [ ]:
import os

row = 2
contrast_images_dir = '/content/drive/MyDrive/fieldbooks_images/1932_road_to_aleppo_12/contrasted/'
for contrasted_image_file_name in os.listdir(contrast_images_dir):
    if contrasted_image_file_name.endswith(".jpg"):
        # Prints only text file present in My Folder
        print(contrasted_image_file_name)
        worksheet_contrasted_pages.update_cell(row, 1, contrasted_image_file_name[7:11])
        worksheet_contrasted_pages.update_cell(row, 3, contrasted_image_file_name[7:-4])        
        text_on_image = get_text_from_image_acs(os.path.join(contrast_images_dir,contrasted_image_file_name))
        worksheet_contrasted_pages.update_cell(row, 5, text_on_image)
        row = row + 1
        print(row,contrasted_image_file_name[7:-4])

# AWS

https://docs.aws.amazon.com/textract/latest/dg/analyzing-document-expense.html

code used from

https://github.com/Raman-Raje/colab_notebooks/blob/master/aws_textract/aws_textract.ipynb
Thanks to : https://github.com/Raman-Raje

In [2]:
# Install what is required
!pip install boto3
!pip install trp
print("restart runtime")

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
restart runtime


In [3]:
# Get keys to connect
import re
import os
import io
import math
import boto3
import pandas as pd
import json

params_directory = "/content/drive/MyDrive/params/" # Change this to suit your machine

params_file_path = os.path.join(params_directory,"params_aws.json")

# Verify the configuration file exists
if(os.path.exists(params_file_path)):
    print("Success.",params_file_path, "exists.")
else:
    print("Failure.",params_file_path, "does not exist.")

# Read the JSON file into a DataFrame
params_dict = pd.read_json(params_file_path, typ='dictionary')

# Store as enivonmental variables
aws_access_key_id = params_dict['your_access_key_id']
aws_secret_access_key = params_dict['your_secret_access_key']
aws_region = params_dict['your_region']

# Do some basic validation
if len(aws_secret_access_key) == 40:
    print("Success. aws_secret_access_key is loaded.")
else:
    print("Error. aws_secret_access_key is not the expected length, please check it.", len(aws_secret_access_key))

Success. /content/drive/MyDrive/params/params_aws.json exists.
Success. aws_secret_access_key is loaded.


In [4]:
#Analyzes text in a document stored in an S3 bucket. Display polygon box around text and angled text 
import boto3
import io
from PIL import Image, ImageDraw
import cv2

def ShowBoundingBox(draw,box,width,height,boxColor):
             
    left = width * box['Left']
    top = height * box['Top'] 
    draw.rectangle([left,top, left + (width * box['Width']), top +(height * box['Height'])],outline=boxColor)   

def ShowSelectedElement(draw,box,width,height,boxColor):
             
    left = width * box['Left']
    top = height * box['Top'] 
    draw.rectangle([left,top, left + (width * box['Width']), top +(height * box['Height'])],fill=boxColor)  

# Displays information about a block returned by text detection and text analysis
def DisplayBlockInformation(block):
    print('Id: {}'.format(block['Id']))
    if 'Text' in block:
        print('    Detected: ' + block['Text'])
    print('    Type: ' + block['BlockType'])
   
    if 'Confidence' in block:
        print('    Confidence: ' + "{:.2f}".format(block['Confidence']) + "%")

    if block['BlockType'] == 'CELL':
        print("    Cell information")
        print("        Column:" + str(block['ColumnIndex']))
        print("        Row:" + str(block['RowIndex']))
        print("        Column Span:" + str(block['ColumnSpan']))
        print("        RowSpan:" + str(block['ColumnSpan']))    
    
    if 'Relationships' in block:
        print('    Relationships: {}'.format(block['Relationships']))
    print('    Geometry: ')
    print('        Bounding Box: {}'.format(block['Geometry']['BoundingBox']))
    print('        Polygon: {}'.format(block['Geometry']['Polygon']))
    
    if block['BlockType'] == "KEY_VALUE_SET":
        print ('    Entity Type: ' + block['EntityTypes'][0])
    
    if block['BlockType'] == 'SELECTION_ELEMENT':
        print('    Selection element detected: ', end='')

        if block['SelectionStatus'] =='SELECTED':
            print('Selected')
        else:
            print('Not selected')    
    
    if 'Page' in block:
        print('Page: ' + block['Page'])
    print()

def get_text_from_image_aws(image_fp):
    client = boto3.client('textract', region_name=aws_region, aws_access_key_id = aws_access_key_id, aws_secret_access_key = aws_secret_access_key)
    with open(image_fp, 'rb') as img_file:
        ### Read bytes ###
        img_bytes = img_file.read()
        response = client.analyze_document(Document={'Bytes': img_bytes}, FeatureTypes=["TABLES", "FORMS"])
    image = Image.open(image_fp)
    #Get the text blocks
    blocks=response['Blocks']
    width, height = image.size
  
    # Create image showing bounding box/polygon the detected lines/text
    text_detected = ""
    for block in blocks:
        #DisplayBlockInformation(block)    
        draw=ImageDraw.Draw(image)
        if 'Text' in block:
            text_detected = text_detected + " "+ block['Text']
        # Draw bounding boxes for different detected response objects
        if block['BlockType'] == "KEY_VALUE_SET":
            if block['EntityTypes'][0] == "KEY":
                ShowBoundingBox(draw, block['Geometry']['BoundingBox'],width,height,'red')
            else:
                ShowBoundingBox(draw, block['Geometry']['BoundingBox'],width,height,'green')             
        if block['BlockType'] == 'TABLE':
            ShowBoundingBox(draw, block['Geometry']['BoundingBox'],width,height, 'blue')
        if block['BlockType'] == 'CELL':
            ShowBoundingBox(draw, block['Geometry']['BoundingBox'],width,height, 'yellow')
        if block['BlockType'] == 'SELECTION_ELEMENT':
            if block['SelectionStatus'] =='SELECTED':
                ShowSelectedElement(draw, block['Geometry']['BoundingBox'],width,height, 'blue')    
    return text_detected


In [5]:
# test the method
text_on_image = get_text_from_image_aws("/content/drive/MyDrive/fieldbooks_images/1932_road_to_aleppo_12/contrasted/ANT_FB_1932-003-0000.jpg")
print(text_on_image)

 ) Juin. C'st uin tanain situi a 1 Est is la Ronte ilst conlyn 1. Fun & burious menime; plants & guelpus togmen me anthoms but jurns it it preneure m y c remi'd mais Plusen two valian, jenvisor 5m A proformance and phri n carrier; men a extail is prems dailler, inth hagment gardent little appenence. he havane mit anition 11 Part L puils verying 1 a maison misine . for mean iean i - - limsu. / it use viamits from h fomells. Nons oftension i init to fani in andry. iproxemate 1 la wonte, , tris it win NO in train - in 10mx cm Commenument 1) havause, " pone. ' 15th le this home Jmm i Om40 2Jum time Avens moraan.un before. D fut in each renisms (ptob pro/mium wuson 260 ) Juin. C'st uin tanain situi a 1 Est is la Ronte ilst conlyn 1. Fun & burious menime; plants & guelpus togmen me anthoms but jurns it it preneure m y c remi'd mais Plusen two valian, jenvisor 5m A proformance and phri n carrier; men a extail is prems dailler, inth hagment gardent little appenence. he havane mit anition 11 Pa

In [ ]:
row = 2
contrast_images_dir = '/content/drive/MyDrive/fieldbooks_images/1932_road_to_aleppo_12/contrasted/'
for row_counter in range(2,25):
    file_num = worksheet_contrasted_pages.get_values("C"+str(row_counter))[0][0]
    print(file_num)
    #ANT_FB_1932-003-0016.jpg
    contrasted_image_file_name = "ANT_FB_" + file_num + ".jpg"
    text_on_image = get_text_from_image_aws(os.path.join(contrast_images_dir,contrasted_image_file_name))
    worksheet_contrasted_pages.update_cell(row_counter, 6, text_on_image)

1932-003-0022


# GCP


Uses Google Coud Vision
Requires a Google Cloud Vision project with billing enabled
https://cloud.google.com/vision/docs/setup

Quick start:
https://cloud.google.com/vision/docs/ocr

In [6]:
# Install what is required
!pip install --upgrade google-cloud-vision
!pip install arabic_reshaper
!pip install bidi
!pip install python-bidi

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 386 kB 4.3 MB/s 
     |████████████████████████████████| 408 kB 51.7 MB/s 
     |████████████████████████████████| 114 kB 46.9 MB/s 
     |████████████████████████████████| 47 kB 4.4 MB/s 
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.17.3
    Uninstalling protobuf-3.17.3:
      Successfully uninstalled protobuf-3.17.3
  Attempting uninstall: google-api-core
    Found existing installation: google-api-core 1.31.6
    Uninstalling google-api-core-1.31.6:
      Successfully uninstalled google-api-core-1.31.6
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.8.2+zzzcolab20220719082949 requires protobuf<3.20,>=3.9.2, but you have protobuf 4.21.5 which is incompatible.
tensorflow-metadata

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: Could not find a version that satisfies the requirement bidi (from versions: none)
ERROR: No matching distribution found for bidi
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [1]:
import os
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = '/content/drive/MyDrive/params/params_gcp.json'

In [2]:
# Code adapted from https://cloud.google.com/vision/docs/ocr  (Thanks for this)
from google.cloud import vision
import io
def get_text_from_image_gcp(path):
    lines_of_text = []  
    """Detects text in the file."""

    client = vision.ImageAnnotatorClient()

    with io.open(path, 'rb') as image_file:
        content = image_file.read()

    image = vision.Image(content=content)
    response = client.text_detection(
    image=image,
    image_context={"language_hints": ["fr"]},  
     )
    
    texts = response.text_annotations
    #print('Texts:')
    for text in texts:
        line_data = []  
        ocr_text = text.description
        line_data.append(text.description)
        
        # store the vertices as a list  x0,y0,x1,y1, etc.
        points = []
        for vertex in text.bounding_poly.vertices:
            points.append(vertex.x)
            points.append(vertex.y)
        line_data.append(points)
        lines_of_text.append(line_data)

    if response.error.message:
        raise Exception(
            '{}\nFor more info on error messages, check: '
            'https://cloud.google.com/apis/design/errors'.format(
                response.error.message))
    
    if(len(lines_of_text)>1):
        #It seems the first entry back from Google contains all the text (multiple lines), I just want each line.
        lines_of_text.pop(0)
    document_text = ""
    for t in lines_of_text:
        document_text = document_text + str(t[0]) + " "
    return document_text, lines_of_text

In [3]:
# test the method
text_on_image = get_text_from_image_gcp("/content/drive/MyDrive/fieldbooks_images/1932_road_to_aleppo_12/contrasted/ANT_FB_1932-003-0000.jpg")
print(text_on_image)

("-1 | Juina C'st un terrain situé à l'Est or La Ronti ; il est conly n l'une if Juniores mecione ; plante de quelqus figniers dink orting bout joins il y preveux ; yo semé de mais . Plusieur tous voisina , d'environ 5on à profondeus , ont pris ! gardend à carrier ; men a extrait is presies desitlers , wind to fragmento belle apparence . Les travaux atét anéli par 11. Pust te punts voisin is la maison voisine a for mureau -11m Su- / sture sécurité pour to fouilles . anday presente Nons oftenons uit os fami de la conte , fis in coin NO in terrain - fon 10m x 8m Commencement of travaux , ce jour , an Ist > Omko de Join home from 2. Jum tarme Divers monnaies , un reniesios ( jetch faforint ) m * bojne foteries orals . proponicur ausen : 200 ", [['-1', [23, 379, 31, 379, 31, 384, 23, 384]], ['|', [125, 32, 139, 32, 139, 56, 125, 56]], ['Juina', [144, 32, 213, 32, 213, 56, 144, 56]], ["C'st", [68, 86, 109, 87, 109, 114, 68, 113]], ['un', [116, 86, 144, 86, 144, 113, 116, 113]], ['terrain', 

In [ ]:
row = 2
contrast_images_dir = '/content/drive/MyDrive/fieldbooks_images/1932_road_to_aleppo_12/contrasted/'
for row_counter in range(2,25):
    file_num = worksheet_contrasted_pages.get_values("C"+str(row_counter))[0][0]
    print(file_num)
    #ANT_FB_1932-003-0016.jpg
    contrasted_image_file_name = "ANT_FB_" + file_num + ".jpg"
    document_text, lines_of_text = get_text_from_image_aws(os.path.join(contrast_images_dir,contrasted_image_file_name))
    print(document_text)
    worksheet_contrasted_pages.update_cell(row_counter, 7, str(document_text))

1932-003-0000
-1 | Juina C'st un terrain situé à l'Est or La Ronti ; il est conly n l'une if Juniores mecione ; plante de quelqus figniers dink orting bout joins il y preveux ; yo semé de mais . Plusieur tous voisina , d'environ 5on à profondeus , ont pris ! gardend à carrier ; men a extrait is presies desitlers , wind to fragmento belle apparence . Les travaux atét anéli par 11. Pust te punts voisin is la maison voisine a for mureau -11m Su- / sture sécurité pour to fouilles . anday presente Nons oftenons uit os fami de la conte , fis in coin NO in terrain - fon 10m x 8m Commencement of travaux , ce jour , an Ist > Omko de Join home from 2. Jum tarme Divers monnaies , un reniesios ( jetch faforint ) m * bojne foteries orals . proponicur ausen : 200 
1932-003-0001
2 wonte lom NE aggiu purn Smily deblais 
1932-003-0002
3 Juin Lm Bo Le travail l'avance à ift . Toymus jes trace de mouss faver palicrement . Le Est a le tanchéi fiergerene . il y avait une poche de fiens ; de fricautions int

# Write HTML files

In [ ]:
# Write the data in the spreadsheet out as html files so it is more browsable.
# Like this:
# https://jeffblackadar.ca/fieldbook_antioch/2_ANT_FB_1932-003-0000.html

import os
row = 2

contrast_images_dir = '/content/drive/MyDrive/fieldbooks_images/1932_road_to_aleppo_12/contrasted/'
for row_counter in range(2,25):
    file_num = worksheet_contrasted_pages.get_values("C"+str(row_counter))[0][0]
    print(file_num)
    #ANT_FB_1932-003-0016.jpg
    contrasted_image_file_name = "ANT_FB_" + file_num + ".jpg"
    print(contrasted_image_file_name)
    html_file = open(os.path.join("/content/drive/MyDrive/fieldbooks_images/1932_road_to_aleppo_12/html/", (str(row_counter)+"_"+contrasted_image_file_name[:-4]+".html")), "w")
    html_file.write('<html>\n<head>\n<meta http-equiv="Content-Type" content="text/html; charset=utf-8">\n<title></title>\n</head>\n<body>\n<table>\n<tr>\n')
    html_file.write('<td>\n')
    html_file.write('<img src="'+contrasted_image_file_name+'">\n<br>\n')
    html_file.write('</td><td>\n')    
    html_file.write('<h2>Azure Cognitive Services</h2>\n<p>\n')
    azure_text = worksheet_contrasted_pages.get_values("E"+str(row_counter))
    for v in azure_text:
        for vs in v:  
            html_file.write(str(vs)+"<br>\n")
    html_file.write('</p>\n')
    html_file.write('<h2>AWS Textract</h2>\n<p>\n')
    aws_text = worksheet_contrasted_pages.get_values("F"+str(row_counter))
    for v in aws_text:
        for vs in v:  
            html_file.write(str(vs)+"<br>\n")
    html_file.write('</p>\n')
    html_file.write('<h2>Google (GCP)</h2>\n<p>\n')
    gcp_text = worksheet_contrasted_pages.get_values("G"+str(row_counter))
    for v in gcp_text:
        for vs in v:  
            html_file.write(str(vs)+"<br>\n")
    html_file.write('</p>\n')    
    print(row,contrasted_image_file_name[7:-4], azure_text, aws_text)
    if row_counter<23:
        next_file_num = worksheet_contrasted_pages.get_values("C"+str(row_counter+1))[0][0]
        next_file_name = str(row_counter+1)+"_"+"ANT_FB_" + next_file_num + ".html"
        html_file.write('Next: <a href="' + next_file_name + '">' + next_file_name + '</a>\n')
    html_file.write('</td>\n')
    html_file.write("</tr>\n</table>\n</body>\n</html>")
    html_file.close()

1932-003-0000
ANT_FB_1932-003-0000.jpg
2 1932-003-0000 [["1 Juin.\nC'est un terrain situé à l'Est or la Route ; il est conlyn\nl'un 1 derrières massimo ; plante de quelpas figuiers - dal\nacting but jeers . it off presseurs ; in ya reme'de mai's.\nPlusieurs tuns voision, d'environ 5m de profondeur, as servi\nÀ carrier ; men a extrait is press dailles, with fragment\ngardent belle apparence. Les travaux net avéli pas 11. Pust\nDe puits voisin A la maison voisine . for nouveau\nSean : - 11mbo . I st use seemite love to fouls .\nNone offirms & droit as faul un povagy, è prosimale\nÀ la conte , pi du coin NO in terrain - sun 10m xIm\nCommencement 1 travaux, 4 por, à Ifhe\n, Om40\n2. Jun\nA frame Divers monacies , une bagno . A foto ries orals\n4.\nremission ( patch fragments) profmeus au son : 200\n"]] [[' ) Juin. C\'st uin tanain situi a 1 Est is la Ronte ilst conlyn 1. Fun & burious menime; plants & guelpus togmen me anthoms but jurns it it preneure m y c remi\'d mais Plusen two valian, 